# Continuous Bag of Words (CBOW)

Sebelum kita membahas lebih dalam mengenai **Continuous Bag of Words (CBOW)**, penting untuk memahami bahwa komputer tidak dapat memahami kata-kata atau bahasa dengan cara yang sama seperti manusia. Kata-kata yang kita gunakan dalam percakapan sehari-hari harus diubah menjadi bentuk numerik agar dapat diproses oleh komputer. Proses ini dikenal sebagai representasi numerik atau *word embeddings*.

---

## Apa Itu Word2Vec?

**Word2Vec** adalah salah satu teknik populer yang digunakan untuk menghasilkan representasi numerik dari kata-kata (*word embeddings*). Word2Vec tidak hanya mengubah kata menjadi angka, tetapi juga memastikan bahwa hubungan semantik antar kata tetap terjaga. Sebagai contoh, kata-kata yang memiliki arti serupa atau sering muncul dalam konteks yang sama (misalnya "raja" dan "ratu") akan memiliki representasi vektor yang mirip dalam ruang multidimensi.

Word2Vec bekerja dengan prinsip bahwa *kata yang muncul dalam konteks yang serupa cenderung memiliki makna yang serupa*. Ada dua pendekatan utama dalam Word2Vec:

### 1. Continuous Bag of Words (CBOW)
Model ini memprediksi kata target berdasarkan kata-kata konteks di sekitarnya. Sebagai contoh, jika Anda memiliki kalimat:  

> "Saya suka belajar Python."

Dengan *window size* 2, CBOW akan menggunakan kata-kata di sekitar ("Saya", "suka", "Python") untuk memprediksi kata target "belajar."

### 2. Skip-gram
Model ini bekerja sebaliknya dari CBOW, yaitu memprediksi kata-kata konteks berdasarkan kata target. Menggunakan contoh yang sama, jika kata target adalah "belajar," Skip-gram akan mencoba memprediksi kata-kata konteks seperti "Saya," "suka," dan "Python."

---

## Contoh Sederhana

Misalkan Anda memiliki kalimat:

> "Kucing duduk di atas tikar."

Untuk komputer memahami kata-kata ini, kita perlu mengubahnya menjadi angka. Langkah-langkahnya adalah sebagai berikut:

### 1. Tokenisasi
Pisahkan kalimat menjadi kata-kata individu:  
`["Kucing", "duduk", "di", "atas", "tikar"]`

### 2. Pembuatan Vektor (One-Hot Encoding)
Setiap kata diwakili oleh vektor biner dalam ruang kata:  
Kucing -> [1, 0, 0, 0, 0] Duduk -> [0, 1, 0, 0, 0] Di -> [0, 0, 1, 0, 0] Atas -> [0, 0, 0, 1, 0] Tikar -> [0, 0, 0, 0, 1]


Vektor ini memungkinkan komputer untuk mengenali hubungan antara kata-kata seperti "kucing" dan "tikar" (mungkin sering digunakan dalam konteks serupa).

---

## Mengapa Word2Vec Penting?

Word2Vec memberikan cara yang efisien untuk menangkap makna semantik dan hubungan antar kata dalam teks. Dengan embedding yang dihasilkan:
- Kata-kata yang semantik serupa seperti "raja" dan "ratu" akan memiliki vektor yang mirip.
- Operasi matematika pada vektor dapat digunakan untuk menemukan hubungan antar kata, seperti:  

Raja - Pria + Wanita ≈ Ratu

--- 

### Import Libraries

In [1]:
import requests
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Lambda
import tensorflow.keras.backend as K

In [2]:
url = "https://www.gutenberg.org/files/1342/1342-0.txt"  # Pride and Prejudice
response = requests.get(url)
text = response.text

In [3]:
def preprocess_text(text):
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    return text

In [4]:
cleaned_text = preprocess_text(text)

In [5]:
sentences = cleaned_text.split('.')

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
word2id = tokenizer.word_index
id2word = {v: k for k, v in word2id.items()}
vocab_size = len(word2id) + 1  # Include padding

In [7]:
sequences = tokenizer.texts_to_sequences(sentences)

In [8]:
def generate_context_target_pairs(sequences, window_size):
    context_length = window_size * 2
    pairs = []
    for seq in sequences:
        for idx, target in enumerate(seq):
            start = max(idx - window_size, 0)
            end = min(idx + window_size + 1, len(seq))
            context = [seq[i] for i in range(start, end) if i != idx]
            pairs.append((context, target))
    return pairs


In [9]:
window_size = 4
pairs = generate_context_target_pairs(sequences, window_size)

In [10]:
context_length = window_size * 2
X = [pad_sequences([context], maxlen=context_length, padding='pre')[0] for context, _ in pairs]
y = [to_categorical(target, num_classes=vocab_size) for _, target in pairs]

X = np.array(X)
y = np.array(y)

In [11]:
model = Sequential([
    tf.keras.Input(shape=(context_length,)),
    Embedding(input_dim=vocab_size, output_dim=100, input_length=context_length),
    Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)),
    Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

C:\Users\62813\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 8, 100)         │       720,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7202)           │       727,402 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,447,602 (5.52 MB)

 Trainable params: 1,447,602 (5.52 MB)

 Non-trainable params: 0 (0.00 B)

None


In [12]:
history = model.fit(X, y, epochs=20, batch_size=32, verbose=1)

Epoch 1/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 46s 11ms/step - accuracy: 0.0417 - loss: 6.8907
Epoch 2/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - accuracy: 0.0673 - loss: 6.1552
Epoch 3/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 44s 11ms/step - accuracy: 0.0890 - loss: 5.8990
Epoch 4/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - accuracy: 0.1152 - loss: 5.5984
Epoch 5/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - accuracy: 0.1400 - loss: 5.3558
Epoch 6/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - accuracy: 0.1580 - loss: 5.1527
Epoch 7/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - accuracy: 0.1736 - loss: 4.9589
Epoch 8/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - accuracy: 0.1897 - loss: 4.7951
Epoch 9/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - accuracy: 0.2042 - loss: 4.6277
Epoch 10/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 44s 11ms/step - accuracy: 0.2154 - loss: 4.4995
Epoch 11/20
3975/3975 ━━━━━━━━━━━━━━━━━━━━ 44s 11ms/step - accuracy: 0.2287 - loss: 4.3576
Epoch 12

In [13]:
embeddings = model.layers[0].get_weights()[0]
print("Embedding shape:", embeddings.shape)

Embedding shape: (7202, 100)


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
def find_similar_words(word, embeddings, word2id, id2word, top_n=5):
    if word not in word2id:
        print(f"Word '{word}' not found in vocabulary!")
        return
    word_vector = embeddings[word2id[word]]
    similarities = cosine_similarity([word_vector], embeddings)[0]
    similar_indices = np.argsort(similarities)[-top_n-1:-1][::-1]
    similar_words = [id2word[idx] for idx in similar_indices]
    return similar_words

In [16]:
similar_words = find_similar_words("love", embeddings, word2id, id2word)
print("Words similar to 'love':", similar_words)

Words similar to 'love': ['conjunction', 'swift', 'lot', 'servility', 'rejoice']
